In [ ]:
import os
import copy

import pandas as pd
import numpy as np
from sklearn import linear_model
import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib qt
pd.set_option('mode.chained_assignment', None)
plt.rc('xtick',labelsize=10)
plt.rc('ytick',labelsize=10)
plt.rc('font', size=10) #controls default text size
plt.rc('axes', titlesize=16) #fontsize of the title
plt.rc('axes', labelsize=15) #fontsize of the x and y labels
plt.rc('legend', fontsize=10)
plt.rcParams["figure.figsize"] = (10,7)

In [ ]:
class Container:

    data: pd.DataFrame = None
    

    def __init__(self, data=None, name=''):
        self.data = data
        self.folder=None
        self.experiment_name = name
        self._log = []
        self.info = {}

    def load_csv(self, path=''):
        path, self.folder, self.experiment_name = self._input_path(path)

        self.data = pd.read_csv(path)
        print(f'Loaded: {self.experiment_name}')
        self.log(f'csv loaded from: {path}')
        return self.data

    def load_hdf5(self, path=''):
        path, self.folder, self.experiment_name = self._input_path(path)

        with pd.HDFStore(path) as file:
            data = file['data']
            meta = file.get_storer('data').attrs.meta
        
        self._log.extend(meta['log'])
        self.log(f'hdf5 loaded from: {path}')
        meta.pop('log')
        self.info.update(meta)
        return data

    def dump(self,**kwargs):
        path = f'{self.folder}\{self.experiment_name}'
        kwargs.update({'log':self.log()})
        with pd.HDFStore(f'{path}.hdf5') as file:
            file.put('data', self.data)
            file.get_storer('data').attrs.meta = kwargs 
                

    def initial_filter(
            self,
            time_in_minutes=False,
            temperature=(-np.inf, np.inf),
            viscosity=(0, np.inf),
            time=(0, np.inf),
    ):
        conditions = dict(
            temperature=temperature,
            viscosity=viscosity,
            time=time,
        )

        temperature_cond = (
            (temperature[0] < self.data['Temperature']) &\
            (self.data['Temperature'] < temperature[1])
            )
        viscosity_cond = (
            (viscosity[0] < self.data['Viscosity']) &\
            (self.data['Viscosity'] < viscosity[1])
            )
        time_cond = (
            (time[0] < self.data['time']) &\
            (self.data['time'] < time[1])
            )

        self.data = self.data[
            temperature_cond &\
            viscosity_cond &\
            time_cond]
        if time_in_minutes: self.data['time'] / 60
        self.log(('initial_filter', conditions))
        return self.data

    def mask_filter(
        self,
        filter_func,
        by='Temperature',
        column='Viscosity',
    ):
        group = self.data.groupby(by=by)[column]
        mask = group.apply(filter_func).droplevel([0]).sort_index().to_numpy()
        self.data = self.data[mask]
        self.log(('mask_filter', filter_func.__name__))
        return self.data

    def linearize(self):
        T_func = lambda t: 1 / (t+273.15)
        v_func = lambda nu: np.log(nu)
        self.data['Temperature'] = self.data['Temperature'].apply(T_func)
        self.data['Viscosity'] = self.data['Viscosity'].apply(v_func)
        self.log('linearize')
        return self.data
    
    def delinearize(self):
        t_func = lambda T: 1 /T - 273.15
        nu_func = lambda v: np.exp(v)
        self.data['Temperature'] = self.data['Temperature'].apply(t_func)
        self.data['Viscosity'] = self.data['Viscosity'].apply(nu_func)
        self.log('delinearize')
        return self.data

    def copy(self):
        new =copy.deepcopy(self)
        return new

    def log(self, msg=None):
        if msg is not None: self._log.append(msg)
        return self._log

    @staticmethod
    def _input_path(path):
        while (path == '') or (not os.path.isfile(path)):
            path = input(f"Input data path: ")
        _path_list = (path).split('\\')
        folder = '\\'.join(_path_list[:-1])
        experiment_name = _path_list[-1].split('.')[0]
        return path, folder, experiment_name


In [ ]:
## Filtration
def z_filter(data: pd.Series):
    mean = data.mean()
    s = data.std(ddof=0)+1e-50
    z_score= np.abs((data-mean) / s) <1
    return z_score

# def z_filter(data: pd.Series):
#     return stats.zscore(data)<1

def whisker_iqr_filter(data: pd.Series):
    whisker_width=0.5
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    iqr = q3 - q1
    return (data >= q1 - whisker_width*iqr) & (data <= q3 + whisker_width*iqr)

def iqr_filter(data: pd.Series):
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    iqr = q3 - q1
    
    return np.abs((data - data.median()) / iqr) < 1

def my_z_filter(data: pd.Series):
    mean = data.median()
    s = data.std()+1e-50
    z_score= np.abs((data-mean) / s) <0.1
    return z_score

# Start

In [ ]:
a = Container()
a.load_csv()
base = a.copy()

## Temporal plots
df =a.data
# df = test_data

fig, ax_v = plt.subplots()
ax_T = ax_v.twinx()

ax_v.scatter(df['time'], df['Viscosity'], color='red', marker='.')
ax_v.set_ylabel('Viscosity', color='red')
ax_v.set_xlabel('time')

ax_T.scatter(df['time'], df['Temperature'], color='blue', marker='.')
ax_T.set_ylabel('Temperature', color='blue')
a.experiment_name

In [ ]:
a.initial_filter(time=(0, np.inf), viscosity=(10, 50), temperature=(12, 42))

In [ ]:
##Save Temporal plots
df =a.data
# df2= base.data
fig, ax_v = plt.subplots()
fig.subplots_adjust(
    top=0.9,
    bottom=0.1,
    left=0.1,
    right=0.9,
    hspace=0.2,
    wspace=0.2,
)
w = a.folder.split('(')[1][:-1]
ax_v.set_title(f'{a.experiment_name}: ({w}% mass)')

ax_T = ax_v.twinx()
ax_v.set_ylabel('Viscosity', color='red')
ax_v.set_xlabel('time')
ax_T.set_ylabel('Temperature', color='blue')

# ax_v.scatter(df2['time'], df2['Viscosity'], color='purple', marker='.')
# ax_T.scatter(df2['time'], df2['Temperature'], color='green', marker='.')

ax_v.scatter(df['time'], df['Viscosity'], color='red', marker='.')
ax_T.scatter(df['time'], df['Temperature'], color='blue', marker='.')

In [ ]:
##Save Temperature plots
df = a.data

fig, ax = plt.subplots()
fig.subplots_adjust(
    top=0.9,
    bottom=0.1,
    left=0.1,
    right=0.9,
    hspace=0.2,
    wspace=0.2,
)
w = a.folder.split('(')[1][:-1]
ax.set_title(f'{a.experiment_name}: ({w}% mass)')

colors= df['Viscosity_verbose'].replace({
    'OK':'g', 'OK_inner':'b', 'image_sweep_check':'r','combine_check':'w',
})
# ax.scatter(x=df2['Temperature'],y=df2['Viscosity'],c= 'black',s=20)
ax.scatter(x=df['Temperature'],y=df['Viscosity'],c= colors,s=5)


sns.lineplot(
    ax=ax,
    data=df,
    x="Temperature",
    y="Viscosity",
    estimator='mean',
    errorbar=("sd", 1),
    # err_style="bars", 
    label ='mean'  
)
sns.lineplot(
    ax=ax,
    data=df,
    x="Temperature",
    y="Viscosity",
    errorbar=('pi',50),
    estimator="median",
    label ='median'  
)

In [ ]:
a.linearize()
a.mask_filter(iqr_filter)
## Temperature plots
df = a.data

fig, ax = plt.subplots()
colors= df['Viscosity_verbose'].replace({
    'OK':'g', 'OK_inner':'b', 'image_sweep_check':'r','combine_check':'w',
})
ax.scatter(x=df['Temperature'],y=df['Viscosity'],c= colors,s=5)

sns.lineplot(
    ax=ax,
    data=df,
    x="Temperature",
    y="Viscosity",
    estimator='mean',
    errorbar=("sd", 1),
    # err_style="bars", 
    label ='mean'  
)
sns.lineplot(
    ax=ax,
    data=df,
    x="Temperature",
    y="Viscosity",
    errorbar=('pi',50),
    estimator="median",
    label ='median'  
)

In [ ]:
##Save fast OLS plot
def regress(data):
    reg = linear_model.LinearRegression(fit_intercept=True)
    X = np.array([data['Temperature']]).T
    Y = np.array(data['Viscosity'])
    reg.fit(X, Y)

    w_T = reg.coef_[0]
    w0_nu = reg.intercept_

    nu0 = np.exp(w0_nu)
    E = 8.314 * w_T / 1000

    def TC_func(t, E=E, nu0=nu0):
        Tt = 1 / (t+273.15)
        return nu0 * np.exp(E / 8.314 * Tt)

    return nu0, E, TC_func


fig, ax = plt.subplots()
fig.subplots_adjust(
    top=0.9,
    bottom=0.1,
    left=0.1,
    right=0.9,
    hspace=0.2,
    wspace=0.2,
)
w = a.folder.split('(')[1][:-1]
ax.set_title(f'{a.experiment_name}: ({w}% mass)')

ax.scatter(a.data['Temperature'], a.data['Viscosity'], color='gray', marker='.')

reg_data = a.data
nu0, EkJ, func = regress(reg_data)
sns.regplot(
    ax=ax,
    data=reg_data,
    x="Temperature",
    y="Viscosity",
    scatter=False,
    truncate=False,
    order=1,
    label=f'All:\nE= {EkJ:5.3f} kJ\nNu0= {nu0:5.3e} cP',
    
)

reg_data = a.data.groupby(by='Temperature')['Viscosity'].median().reset_index()
nu0, EkJ, func = regress(reg_data)
sns.regplot(
    ax=ax,
    data=reg_data,
    x="Temperature",
    y="Viscosity",
    scatter=False,
    truncate=False,
    order=1,
    label=f'Median:\nE= { EkJ:5.3f} kJ\nNu0= {nu0:5.3e} cP',
)

reg_data = a.data.groupby(by='Temperature')['Viscosity'].mean().reset_index()
nu0, EkJ, func = regress(reg_data)
sns.regplot(
    ax=ax,
    data=reg_data,
    x="Temperature",
    y="Viscosity",
    scatter=False,
    truncate=False,
    order=1,
    label=f'Mean:\nE= { EkJ:5.3f} kJ\nNu0= {nu0:5.3e} cP',
)
plt.legend()

In [ ]:
results = pd.DataFrame(columns=['E_kJ','dE_kJ','nu0_cP','dnu0_cP','name','w','desc'])

In [ ]:
## Regression
reg_a = a.copy()

df = reg_a.data
df.rename(inplace=True, columns={'Temperature': 'E', 'Viscosity': 'nu'})
df['nu0'] = 1
result_OLS = sm.OLS(df['nu'], df[['E', 'nu0']]).fit()
means = result_OLS.params
conf_int = result_OLS.conf_int(0.005).loc

conf_int['nu0'] = np.exp(conf_int['nu0'])
nu0 = np.exp(means['nu0'])
nu0_diap = (conf_int['nu0'].max() - conf_int['nu0'].min()) / 2
nu0_power = round(np.log10(nu0_diap))

conf_int['E'] = 8.314 * conf_int['E'] / 1000
EkJ = 8.314 * means['E'] / 1000
EkJ_diap = (conf_int['E'].max() - conf_int['E'].min()) / 2
EkJ_power = round(np.log10(EkJ_diap))+1

exp_name = reg_a.experiment_name
w = reg_a.folder.split('(')[1][:-1]
print(
    f'Constants {exp_name} ({w}% mass):',
    f'E   = {EkJ/10**EkJ_power: <10.3f} ± {EkJ_diap/10**EkJ_power: <3.2f} kJ *1e{EkJ_power}',
    f'Nu0 = {nu0/10**nu0_power: <10.3f} ± {nu0_diap/10**nu0_power: <3.2f} cP *1e{nu0_power}',
    sep='\n',
)
print(result_OLS.summary2())

In [ ]:
## Save result
desc= input('Description')
results.loc[len(results)] = [
    EkJ,
    EkJ_diap,
    nu0,
    nu0_diap,
    reg_a.experiment_name,
    w,
    desc
]
reg_a.dump(**results.loc[len(results)-1].to_dict())
results

In [ ]:
results.to_excel(r'Experiments\MultiplyTemperature\Results.xlsx')

In [ ]:
y=[2.5,3.7,6,8]
X= [1,2,3,4]
plt.scatter(X,y)
result_OLS = sm.OLS(y, X).fit()
print(result_OLS.summary2())